In [4]:
import numpy as np
import pandas as pd
import scanpy as sc
h5ad_file_dir  = '../database/celltypist/CellTypist_training_data_for_cellxgene.h5ad'
adata = sc.read(h5ad_file_dir,cache=True)
ligation_receptor_dir = '../database/data/BioMap_Ligand_Receptor_v1.tsv'
ligation_receptor_df = pd.read_csv(ligation_receptor_dir, sep='\t')
ligation_df = ligation_receptor_df['ligand']
receptor_df = ligation_receptor_df['receptor']

In [44]:
from tqdm import tqdm
result = pd.DataFrame(columns = ['cell_type','gene_symbol','is_ligand/receptor','count'])
#adata_test = adata[:5,['5S_rRNA','A1BG','A1BG-AS1','A1CF','A2M','A2M-AS1','A2ML1','A2ML1-AS1','A2ML1-AS2','A2MP1']]
gene_map = pd.concat([ligation_df,receptor_df]).drop_duplicates()
gene_map_valid = [k for k in gene_map if k in adata.var_names]
adata_test = adata[:, gene_map_valid]

# adata_test = adata
# (adata_test.to_df()).head()
celltypes = (adata_test.obs['Harmonised_detailed_type']).drop_duplicates()
m = 0
k = 0
wrong = pd.DataFrame(columns = ['cell_type','gene_symbol','is_ligand/receptor','count'])
for i in celltypes:
    #i = celltypes[w]
    bdata =  adata_test[adata_test.obs.Harmonised_detailed_type == i]
    cdata = bdata.to_df() ##行 不重复的sample，列基因，内容就是count
    for u in tqdm(range(len(adata_test.var_names))):## j = 5S_rRNA:
        j = adata_test.var_names[u]
        count = float(cdata[j].median())
        if j in list(receptor_df):
            if j in list(ligation_df):
                result.loc[m] = [i,j,'receptor',count]
                result.loc[m+1] = [i,j,'ligand',count]
                m = m+2
                k = k
            else:
                result.loc[m] = [i,j,'receptor',count]
                m=m+1
                k = k
        else:
            if j in list(ligation_df):
                result.loc[m] = [i,j,'ligand',count]
                m = m+1
                k = k
            else:
                #wrong.loc[k] = [i,j,'wrong',count]
                k = k+1
                m = m
result.to_csv('../project/celltypist/singlecell-task2.tsv',sep='\t',index = False)
#wrong.to_csv('../project/singlecell-task2-bugproteinname.tsv',sep='\t')
gene_map

100%|███████████████████████████████████████| 3185/3185 [02:36<00:00, 20.31it/s]


0            A2M
1          AANAT
3            ACE
5         ADAM10
7         ADAM12
          ...   
55626    SIGLEC9
55661      RNF43
55665    SIGLEC5
55669     PLXDC2
55701       VASN
Length: 3209, dtype: object

In [87]:
bdata = adata[adata.obs.Harmonised_broad_type == 'Macrophages']
cdata = bdata[:5].to_df()
count = cdata['5S_rRNA'].median()
count

0.0

In [1]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [45]:
len(celltypes)

90

In [46]:
result.to_csv('../project/celltypist/singlecell-reform.tsv',sep='\t',index = False)